In [ ]:
import math
import json
import re
import copy

import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")

# Load an example dataset with long-form data
#fmri = sns.load_dataset("fmri")

# Plot the responses for different events and regions
#plt = sns.lineplot(x="timepoint", y="signal",
#             hue="region", style="event",
#             data=fmri)


#plt.show()

In [ ]:

#with open('../1_pop_cleaning/communes_VD_clean.json', 'r') as cf:
#    communes = json.load(cf)
#    for c in communes:
#        for hy in c["hab_year"]:
#            hy["pop"] = int(hy["pop"])
#            hy["year"] = int(hy["year"])
#cdata = pd.DataFrame(communes)
#cdata

In [ ]:
def interpolator(data_points):
    """Returns a linear interpolator from the given dataPoints
    @param {*} dataPoints an array of length 2 arrays, each sub-array is a coordinate with sub-array[0]=x, sub-array[1]=y
    @returns interpolate(x) a function taking a value x and returning the linear interpolation of y at x, or null if x is outside the x range of dataPoints
    """
    data_points = sorted(data_points, key=lambda hy:hy[0])
    if len(data_points)<=1:
            return lambda x: None
    else:
        def interpolate(x):
            if data_points[0][0]==x:
                return data_points[0][1]
            for i in range(1,len(data_points)):
                if data_points[i][0]>=x and data_points[i-1][0]<=x:
                    a = data_points[i-1]
                    b = data_points[i]
                    return a[1]+ (b[1]-a[1])/(b[0]-a[0]) * (x-a[0])
            return None
        return interpolate
    

In [ ]:
def exponential_interpolator(data_points):
    linear_interpolator = interpolator([(dp[0],math.log(dp[1])) for dp in data_points])
    def exp_interpolate(x):
        log_result = linear_interpolator(x)
        if log_result:
            return math.exp(log_result)
    return exp_interpolate

In [ ]:
a = [(0.01,0.1),(1,1),(2,3),(3,6)]
print(f"a = {a}")
x=2.5
interp = interpolator(a)
print(f"interp({x}) = {interp(x)}")
expterp = exponential_interpolator(a)
print(f"expterp({x}) = {expterp(x)}")

In [ ]:
class HabYear:
    UNIT_EINW_1 = ['Beisassen',
    'Bürger',
    'Einw',
    'Einw Feuerschaukreis',
    'Einw Kirchgem.',
    'Einw. Kirchgem', 'Einwohner',
    'Kirchgenossen', 'Kommunikanten'
    ]
    UNIT_HAUSHALTE_5 = [
    'Erwachsene', 'Erwachsenen', 'Fam', 'Feuerstellen', 'Feuerstätten',
    'Feuerstätten (mit Sermuz, heute Gem', 'Haushalt', 'Haushalte',
    'Haushalte Kirchgem', 'Haushaltungen', 'Hausväter', 'Herdstellen',
    'Herdstätten', 'Hofgenossen', 'Hofstätten', 'Häuser', 'Höfe',
    'Steuerpflichtige',
    ]
    ACCEPTED_UNITS = UNIT_EINW_1+UNIT_HAUSHALTE_5
    
    def __init__(self, year, pop, unit=None, type_="original"):
        self.year = int(year) 
        self.pop = int(pop)
        self.unit = unit.strip() if unit else "undefined"
        self.type = type_
    def to_json(self):
        json = copy.deepcopy(self.__dict__)
        if "next" in json and json["next"]:
            #del json["next"]
            json["next"] = json["next"].year
        if "previous" in json and json["previous"]:
            #del json["previous"]
            json["previous"] = json["previous"].year
        return json
    def __str__(self):
        return self.to_json().__str__()
    def __repr__(self):
        return self.__str__()
    def convert_unit_to_Einw(self, conversion_factor):
        if self.unit in HabYear.UNIT_HAUSHALTE_5:
            self.original_unit = self.unit
            self.einw_conversion_factor = conversion_factor
            self.pop = int(np.round(conversion_factor*self.pop))
        self.unit = "Einw"
    def compute_growth_rate(self):
        # g^dy p1 = p2 -> g = (p2/p1)^(1/dy) -> log(g) = (log(p2)-log(p1))/dy
        if self.next:
            dy = self.next.year - self.year
            self.ygr = (self.next.pop / self.pop) ** (1/dy)
    @staticmethod
    def interpolator(interp, unit="Einw"):
        def hy_interp(year):
            pop = interp(year)
            if pop:
                return HabYear(year, pop, unit, "interpolated")
        return hy_interp

In [ ]:

            
        
class Commune:
    COMMUNES_NAME_REGEX = re.compile(r"\W\(?Gemeinde\)?")
    def __init__(self, json_commune):
        self.name = self.COMMUNES_NAME_REGEX.sub("",json_commune["name"])
        self.canton = json_commune["canton"]
        self.url = json_commune["url"]
        self.firstmention = json_commune["firstmention"] if "firstmention" in json_commune else None
        self.notes = json_commune["notes"] if "notes" in json_commune else ""
        self.hab_year = [HabYear(hy["year"], hy["pop"], hy.get("unit"), "original") for hy in json_commune["hab_year"]]
        self.raw_hab_year = copy.deepcopy(json_commune["hab_year"])
    def to_json(self):
        json = copy.deepcopy(self.__dict__)
        json["hab_year"] = [hy.to_json() for hy in json["hab_year"]]
        return json
    def __str__(self):
        return self.to_json().__str__()
    def __repr__(self):
        return self.__str__()
    def has_pop_data(self,year):
        return year in [hy.year for hy in self.hab_year]
    def order_hab_year(self):
        if len(self.hab_year)>1:
            self.hab_year = sorted(self.hab_year,key=lambda hy: hy.year)
            for i in range(1,len(self.hab_year)):
                self.hab_year[i].previous = self.hab_year[i-1]
                self.hab_year[i-1].next = self.hab_year[i]
            # for original hy: next&previous are only original ones ;-)
            original_hab_years = [hy for hy in self.hab_year if hy.type=="original"]
            for i in range(1,len(original_hab_years)):
                original_hab_years[i].previous = original_hab_years[i-1]
                original_hab_years[i-1].next = original_hab_years[i]
        if len(self.hab_year)>0:
            self.hab_year[0].previous = None
            self.hab_year[len(self.hab_year)-1].next = None
    def detect_duplicated_years(self):
        year_dict = {}
        for hy in self.hab_year:
            if hy.year in year_dict:
                year_dict[hy.year].append(hy)
            else:
                year_dict[hy.year] = [hy]
        duplicates = {y:sorted(hys,key=lambda hy: - hy.pop) for y,hys in year_dict.items() if len(hys)>1}
        return duplicates
    def remove_duplicates(self):
        duplicates = self.detect_duplicated_years()
        self.hab_year = [hy for hy in self.hab_year if hy.year not in duplicates or (hy.unit=="Einw" and self.name not in ["Därstetten", "Egerkingen"])]
    def remove_non_accepted_units(self):
                self.hab_year = [hy for hy in self.hab_year if hy.unit in HabYear.ACCEPTED_UNITS]
    def convert_hab_year_to_Einw(self, conversion_factor):
        for hy in c.hab_year:
            hy.convert_unit_to_Einw(conversion_factor)
    def create_hab_year_interpolator(self, years):
        self.order_hab_year()
        interp_space = [(hy.year,hy.pop) for hy in self.hab_year if hy.type=="original"]
        self.interpolator = exponential_interpolator(interp_space)
        self.hy_interpolator = hy_interpolator(self.interpolator)
    def interpolate_hab_year(years):
        for hy in c.hab_year:
            hy.compute_growth_rate()
        for y in years:
        hy = self.hy_interpolator(y)
        if hy and not self.has_pop_data(y) :
            self.hab_year.append(hy)
        #print(f"{i} {c.name}")
        c.order_hab_year()
        

# Loading data & assembling reviewedCommunes with non-reviewed communes

In [ ]:

#with open('../1_pop_cleaning/communes_V2_checkpoint_1555579436131.json', 'r') as cf:
with open('../1_pop_cleaning/communes_V2_checkpoint_1558469560478.json', 'r') as cf:
    data = json.load(cf)
    

In [ ]:

reviewedCommunes = data["reviewedCommunes"]
communesToReview = data["communesToReview"]

# remember that some are reviewed and some not
for commune in reviewedCommunes:
    commune["hand_reviewed"] = True
for commune in communesToReview:
    commune["hand_reviewed"] = False
    
print("communes[0].keys()")
print(reviewedCommunes[0].keys())
print("communesToReview[0].keys()")
print(communesToReview[0].keys())

In [ ]:
# class version

columns_communes = ["name","canton","url","firstmention","hab_year","raw_hab_year","notes"]

communes = reviewedCommunes+communesToReview
communes = [Commune(json_commune)
    for json_commune in communes
]

# Drop thurgau for now
print(len(communes))
TGcommunes = [c for c in communes if c.canton=="TG"]
communes = [c for c in communes if c.canton!="TG"]
print(len(communes))

#dfcommunes = pd.DataFrame(communes)[columns_communes]

#pd.set_option('display.max_rows', None) 
#dfcommunes

In [ ]:
for c in communes:
    if c.name == "Därstetten":
        print("WHY DÄRSTETTEN?!?!! WHYY!!?")

# Available data across time

In [ ]:
years_bins = range(1150,1901,50)

years = [hy.year for c in communes for hy in c.hab_year if hy.year<=1850]
print("min(years) = ", min(years))

print("Number of observations per year:")
print(pd.Series(years).value_counts(bins=years_bins,sort=False))

sns.distplot(years, bins=years_bins, kde=False, rug=False).set_title('Number of observations across time')

In [ ]:


first_years = [min([y for hy in c.hab_year for y in [hy.year] if y<=1850]+[1850]) for c in communes ]

sns.distplot(first_years, bins=years_bins, hist_kws=dict(cumulative=True),kde=False,rug=False).set_title("Number of communes with first observation before year Y")

# Unit transformation

In [ ]:
units = [hy.unit  for c in communes for hy in c.hab_year if hy.year<1850]

print("Number of each type of units:")
print(pd.Series(units).value_counts())

In [ ]:
pd.Series(units).unique()

### Finding the conversion factor

In [ ]:

def one1one5(a,b):
    return (a.unit in HabYear.UNIT_EINW_1 and b.unit in HabYear.UNIT_HAUSHALTE_5) or (a.unit in HabYear.UNIT_HAUSHALTE_5 and b.unit in HabYear.UNIT_EINW_1)


# find duplicates
duplicates = [(c.name,c.canton,y,hys) for c in communes for y,hys in c.detect_duplicated_years().items() if one1one5(hys[0],hys[1])]

dfduplicates = pd.DataFrame([{
    "commune": cname,
    "canton": canton,
    "year": y,
    "pop_einw": hys[0].pop,
    "pop_haus": hys[1].pop,
    "ratio": hys[0].pop / hys[1].pop,
    "unit_haus": hys[1].unit} for cname, canton, y, hys in duplicates])
dfduplicates = dfduplicates.sort_values(["year","unit_haus","commune"])
print("Number of double Haushaltungen-Einw data")
print(len(duplicates))
dfduplicates

In [ ]:
def boxplot_condition(df, groupings, ax=plt):
    groupings = [(unit, f, f(dfduplicates)) for unit, f in groupings]
    ax.boxplot(
        [df.ratio.loc[condition] for unit, f, condition  in groupings],
        labels = [unit+f" ({np.sum(condition)}, mean: {np.round(np.mean(df.ratio.loc[condition]),2)})" for unit, f, condition in groupings]
        )
    #ax.xticks(rotation=10)
    ax.tick_params(labelrotation=10)

    

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,4.8))

haus_units = [
    ("all", lambda df: df.pop_haus>0),
    ("Haushalte", lambda df: (df.unit_haus =="Haushaltungen") | (df.unit_haus=="Haushalte")),
    ("Häuser", lambda df: df.unit_haus=="Häuser"),
    ("Feuerstätten", lambda df: df.unit_haus=="Feuerstätten"),
    ("Steuerpflichtige", lambda df: df.unit_haus=="Steuerpflichtige")
]

haus_years= [
    ("all", lambda df: df.year>=0),
    (">1750", lambda df: df.year>1750),
    ("1650-1749", lambda df: (df.year>1650) & (df.year<=1750)),
    ("<1650", lambda df:  df.year<=1650),
]

haus_cantons = [
    ("all", lambda df: df.pop_haus>0),
    ("VD", lambda df: df.canton =="VD"),
    ("others", lambda df: df.canton !="VD"),
]

boxplot_condition(dfduplicates, haus_units, ax1)
ax1.set_title("conversion factor boxplot depending on type of unit")
boxplot_condition(dfduplicates, haus_years, ax2)
ax2.set_title("conversion factor boxplot depending on year of observation")
boxplot_condition(dfduplicates, haus_cantons, ax3)
ax3.set_title("conversion factor boxplot depending on canton")
None

In [ ]:
conversion_factor = np.mean(dfduplicates.ratio)
print(f"conversion factor: {np.round(conversion_factor,3)}")

### Doing the conversion

Remove non-Einw data in duplicated years, as well as non-accepted units

In [ ]:
for c in communes:
    c.remove_duplicates()
    c.remove_non_accepted_units()
    c.convert_hab_years_to_Einw(conversion_factor)

In [ ]:
c = communes[9]
duplicates = c.detect_duplicated_years()
c.hab_year

In [ ]:
units = [hy.unit  for c in communes for hy in c.hab_year if hy.year<1850]

print("Number of each type of units:")
print(pd.Series(units).value_counts())

In [ ]:
dfcommunes = pd.DataFrame([c.to_json() for c in communes])[columns_communes]

#pd.set_option('display.max_rows', None) 
dfcommunes

In [ ]:
dfcommunes.to_csv("communes_units_converted.csv", sep=";")

In [ ]:
with open('communes_units_converted.json', 'w') as json_file:
  json.dump([c.to_json() for c in communes], json_file)

In [ ]:
communes[0]

# Doing the interpolation

Create an interpolator per commune:

add hab_year interpolator to each commune:

In [ ]:
for c in communes:
    c.create_hab_year_interpolator()
    

## How to extrapolate for communes without data?

### options:
- compute year-by-year growth rates for communes with data
   -> use the average per-year growth rate to find a growth-rates time series back in time
   -> use this growth-rate time series to go back in time
- use timespans of size X: compute average growth rate in each timespan
    -> go back in time with those growth rates
    
### proposition:
- each hy has a "type" entry with possible values: "original", "interpolated", "extrapolated"
- define function compute_growth_rate(c,y1,y2):
    return None if no data surrounding y1, y2
    return growth rate if it exists
- iterate over communes and timespans
    compute_growth_rate(...)
    -> growth rate of timespan = avg of computed growth rate
    
## TODO:
- compute_growth((x1,y1),(x2,y2))
- compute_growth(c,y1,y2)

In [ ]:
all_years = np.unique(sorted(years))

compute all the intermediate year:

In [ ]:
for c in communes:
    for y in all_years:
        hy = c.hy_interpolator(y)
        if hy and not c.has_pop_data(y) :
            c.hab_year.append(hy)
    #print(f"{i} {c.name}")
    c.order_hab_year()
    for hy in c.hab_year:
        hy.compute_growth_rate()
            

In [ ]:
communes[0].hab_year

# TEST

In [ ]:
communes[32].order_hab_year()

In [ ]:
hys = communes[32].hab_year
hys

In [ ]:
sorted(hys,key=lambda hy: hy.year)

In [ ]:
hys[0].next

In [ ]:
hys[0].to_json()

In [ ]:
hys[0].next

In [ ]:
hys[0].previous

In [ ]:
da = {"hu":32, 55:123}
da

In [ ]:
del da["hu"]

In [ ]:
da

In [ ]:
json = copy.deepcopy(hys[1].__dict__)
json

In [ ]:
"next" in json

In [ ]:
del json["next"]

In [ ]:
json

In [ ]:
if "previous" in json:
    del json["previous"]
json

In [ ]:

if "next" in json:
    del json["next"]
if "previous" in json:
    del json["previous"]

In [ ]:
hys[2].to_json()